In [ ]:
import pandas as pd
import numpy as np

## Worklist prioritization: Emergency Setting

In [ ]:
## First, read in the file of the current worklist with the probabilities that your two algorithms have
## generated for the two types of findings you're most concerned with:

worklist = pd.read_csv('probabilities.csv')

In [ ]:
worklist.head()